## Web Scraping, API's, Pandas

This Workbook contains exercises by NOVA School of Business and Economics a friend asked me to explain to him. I wrote the code and tutored him for it.

### Question 1 (30 points) Movies Database

![](https://storage.googleapis.com/kaggle-datasets-images/1474/2639/cb50fd3b236d01184fd23ad59af50b4f/dataset-original.jpg)



IMDb (also known as the Internet Movie Database) is an online database of information related to films, television programs, home videos, video games, and streaming content online – including cast, production crew and personal biographies, plot summaries, trivia, ratings, and fan and critical reviews.

As of January 2020, IMDb has approximately 6.5 million titles (including episodes) and 10.4 million personalities in its database, as well as 83 million registered users.


Consider the top 250 rated movies on IMDb: https://www.imdb.com/chart/top/ 
 


**Question 1.1** (10 points) Webscrape the mentioned page and retrieve the mean rated value for the top 100 movies.

In [34]:
#Question 1.1
import requests
from bs4 import BeautifulSoup
import statistics
 
url = "https://www.imdb.com/chart/top/"
 
list_of_titles = []
list_of_years = []
user_ratings = []

r = requests.get(url)

rawdata = requests.get(url).text
movies = BeautifulSoup(rawdata, 'lxml')
 
tbody = movies.tbody
table_rows = tbody.find_all('tr')
 
for td in table_rows:
    imdb_ratings = td.find_all('td', class_="ratingColumn imdbRating")
    
    for rating in imdb_ratings:
        user_rating = rating.text.strip()
        user_ratings.append(float(user_rating))
        
length = len(user_ratings)
print("Length test: {}".format(length))
print("The mean rating for the TOP100 movies is: {}".format(round((statistics.mean(user_ratings[:100])),50)))



Length test: 250
The mean rating for the TOP100 movies is: 8.477


**Question 1.2** (20 points) Webscrape the mentioned page and create a `CSV` named `top100movies.csv` with `Rank`, `Orignal Title`, `Rate`, `Year`, `Duration` (in minutes), `Budget` in millions and return of investment, `ROI`, calculated from the "Cumulative Worldwide Gross", for the top 100 movies. 

See the example below:

 | Rank | Original Title                 | Rate  | Year  | Duration  | Budget | ROI |
| ------------- |:-------------:| :-------------:| :-------------:|  :-------------:|   :-------------:|   ------------:|
| 1 | The Shawshank Redemption      | 9.3 | 1994 | 142  | 25.0M | 15.26%  |

In [35]:
# question 1.2
import requests
from bs4 import BeautifulSoup
import statistics
 
url = "https://www.imdb.com/chart/top/"
 


In [36]:
user_ratings = []
rank = []
original_title = []
year = []
hyperlinks = []
 
rawdata = requests.get(url).text
movies = BeautifulSoup(rawdata, 'lxml')
 
tbody = movies.tbody
table_rows = tbody.find_all('tr')
 
for td in table_rows:
    imdb_ratings = td.find_all('td', class_="ratingColumn imdbRating")
    
    for rating in imdb_ratings:
        user_rating = rating.text.strip()
        user_ratings.append(user_rating)
        
    for r in td.find_all('td',class_= "titleColumn"):
        titlestuff = r.text.strip().split('\n')
        #Rank
        rank.append(titlestuff[0][:-1])
        #Original Title
        original_title.append(titlestuff[1].strip())
        #Year
        year.append(titlestuff[2][1:-1])
        
    for i in td.find_all('a'):
        hyperlink = (i.get('href'))
        hyperlinks.append('https://www.imdb.com' + hyperlink)
hyperlinks = hyperlinks[::2][:100]


In [37]:
#These lines need some minutes
single_page_soup = []

for i in hyperlinks[:100]:
    single_page_soup.append(BeautifulSoup(requests.get(i).text,'lxml'))
    

In [38]:
minutes = []
budget = []
return_on_invest = []
for page in single_page_soup:
    #Minutes
    minutes.append(page.find(text='Runtime:').findNext('time').text.strip().split()[0])
    #Budget
    try:
        bud = page.find(text='Box Office').findNext('div').text.strip().split('$')[1].split('\n')[0]
        bud = float(bud.replace(',',''))
        budget.append(str(round(bud/1000000,2)) + "M")
    except Exception:
        bud = "N/A"
        budget.append(bud)
    #ROI
    try:
        revenue = page.find(text='Cumulative Worldwide Gross:').parent.parent.text.strip().split('$')[1]
        revenue = float(revenue.replace(',',''))
        roi = str(round((revenue/bud-1)*100,2)) + '%'
        return_on_invest.append(roi)
    except Exception:
        roi = 'N/A'
        return_on_invest.append(roi)

In [39]:
import csv

with open('top100movies.csv', 'w', newline='') as f:
    thewriter = csv.writer(f)
    
    thewriter.writerow(['Rank','Original Title','Rate','Year','Duration', 'Budget','ROI'])
    
    for i in range(0,100):
        thewriter.writerow([rank[i],original_title[i], user_ratings[i], year [i],minutes [i], budget [i], return_on_invest[i]])

<br><br>

### Question 2 (15 points) ISS Location

![](https://www.esa.int/var/esa/storage/images/esa_multimedia/images/2011/02/international_space_station/17901086-3-eng-GB/International_Space_Station_pillars.jpg)


The International Space Station (ISS) is a modular space station (habitable artificial satellite) in low Earth orbit. 

The ISS location can be retrieved with the following API: http://open-notify.org/Open-Notify-API/ISS-Location-Now/

<br>


**Question 2.1** (5 points) Use the API mentioned to get the location of the ISS. Print the latitude, the longitude and time in the format YYYY-MM-DD HH:MM:SS

Hint: You can use the python `datetime` module (https://docs.python.org/3/library/datetime.html) to read the timestamp to a datetime object with the following code:

```python
from datetime import datetime

timestamp = 1575750073
dt_object = datetime.fromtimestamp(timestamp)
```

In [40]:
import requests
import json

In [41]:
# question 2.1
from datetime import datetime



iss = requests.get('http://api.open-notify.org/iss-now.json')

iss = iss.json()
ts = iss['timestamp']
long = iss['iss_position']['longitude']
lat = iss['iss_position']['latitude']

timestamp = ts
dt_object = datetime.fromtimestamp(timestamp)
dt_object

print(dt_object)
print(lat)
print(long)

2021-03-27 17:24:56
37.2333
97.1999


**Question 2.2** (10 points) Make at least two requests to this API with a time interval of 10 seconds. Print at which speed in Km/s the ISS travelled.

<br>

Hint 1: To calculate the distance from two geocoordinates we recommend the `haversine` library that must be installed with `pip install haversine`. Documentation:  https://pypi.org/project/haversine/

<br>

Hint 2: To sleep your program you can use the `time` python module:

```python
import time

print("Hello")
time.sleep(4.1)
print("Hello after 4.1 seconds")
```

In [42]:
pip install haversine

Note: you may need to restart the kernel to use updated packages.


In [43]:
import time 
from haversine import haversine
timest = []
lati = []
longi = []

In [44]:
timest = []
lati = []
longi = []
for i in range(0,2):
    iss = requests.get('http://api.open-notify.org/iss-now.json')

    iss = iss.json()
    ts = iss['timestamp']
    long = iss['iss_position']['longitude']
    lat = iss['iss_position']['latitude']

    timestamp = ts
    dt_object = datetime.fromtimestamp(timestamp)
    
    timest.append(dt_object)
    lati.append(float(lat))
    longi.append(float(long))
    
    time.sleep(10)
print(timest[1]-timest[0])

0:00:11


In [45]:
one = (lati[0],longi[0])
two = (lati[1],longi[1])
km_per_10sec = haversine(one,two)
km_h = km_per_10sec*6*60

In [46]:
km_h

26169.611858166758

In [47]:
print('The ISS moves at a speed of ' + str(km_h) + 'km/h.')

The ISS moves at a speed of 26169.611858166758km/h.


<br><br>

### Question 3 (40 points) Aircraft Wildlife Strikes

![](https://i.dailymail.co.uk/1s/2020/07/07/16/30492678-0-image-a-15_1594136775641.jpg)

The "wildlifestrikes.csv" dataset contains a record of each reported wildlife strike of a military, commercial, or civil aircraft between 1990 and 2015. Each row contains the incident date, aircraft operator, aircraft make and model, engine make and model, airport name and location, species name and quantity, and aircraft damage.

The wildlife strike database was compiled from reports received from airports, airlines, and pilots and published by the Federal Aviation Association.
<br>

**Question 3.1** (2.5 points) Show the number of records, the number of columns and all column names in the dataset.

In [60]:
# question 3.1
import pandas as pd

df = pd.read_csv('faa-wildlife-strikes.csv')
#df.info()
df.head()

,INDEX_NR,OPID,OPERATOR,ATYPE,AMA,AMO,EMA,EMO,AC_CLASS,AC_MASS,...,REPORTED_NAME,REPORTED_TITLE,REPORTED_DATE,SOURCE,PERSON,NR_INJURIES,NR_FATALITIES,LUPDATE,TRANSFER,INDICATED_DAMAGE
0,100000,AAL,AMERICAN AIRLINES,B-727,148,10,34.0,10,A,4.0,...,Deleted,Deleted,1992-05-14 00:00:00,FAA Form 5200-7,Pilot,NaN,NaN,1996-03-14 00:00:00,0,0
1,100001,UAL,UNITED AIRLINES,B-737-300,148,24,10.0,01,A,4.0,...,Deleted,Deleted,NaN,Air Transport Report,Air Transport Operations,NaN,NaN,2007-12-20 00:00:00,0,0
2,100002,UAL,UNITED AIRLINES,B-737-300,148,24,10.0,01,A,4.0,...,Deleted,Deleted,NaN,Air Transport Report,Air Transport Operations,NaN,NaN,2007-12-20 00:00:00,0,0
3,100003,UAL,UNITED AIRLINES,B-757-200,148,26,34.0,40,A,4.0,...,Deleted,Deleted,NaN,Air Transport Report,Air Transport Operations,NaN,NaN,2007-12-20 00:00:00,0,0
4,100004,UAL,UNITED AIRLINES,A-320,04A,03,23.0,01,A,4.0,...,Deleted,Deleted,NaN,Air Transport Report,Air Transport Operations,NaN,NaN,2007-12-20 00:00:00,0,0


**Question 3.2** (2.5 points) How many different species the dataset has? Consider the `Species ID` column for this exercise. You can consider the different unknown species as unique values.

In [49]:
# question 3.2
len(set(df['SPECIES_ID']))

719

**Question 3.3** (5 points) Show the number of records in percentage by month.

In [50]:
# question 3.3
df1 = df.groupby(['INCIDENT_MONTH']).count()['INDEX_NR'].apply(lambda x: x/df['INDEX_NR'].count()*100)
df1

INCIDENT_MONTH
1      3.491017
2      3.209002
3      5.022860
4      7.287024
5      9.688462
6      8.316294
7     12.675183
8     14.068603
9     13.574071
10    12.035335
11     6.662684
12     3.969467
Name: INDEX_NR, dtype: float64

**Question 3.4** (10 points) Show the number of records, injuries and fatalities by year. Consider "na" values as 0.

In [51]:
# question 3.4

a = df['INDEX_NR'].groupby(df['INCIDENT_YEAR']).count()
b = df['NR_INJURIES'].groupby(df['INCIDENT_YEAR']).sum()
c = df['NR_FATALITIES'].groupby(df['INCIDENT_YEAR']).sum()
z = df['INCIDENT_YEAR'].unique()

df2 = pd.DataFrame(list(zip(z,a,b,c)),columns=['Year','Number of Records', 'Injuries', 'Fatalities'])
df2

,Year,Number of Records,Injuries,Fatalities
0,1992,2099,8.0,0.0
1,1996,2672,11.0,0.0
2,1991,2816,8.0,1.0
3,1993,2849,6.0,1.0
4,1994,2938,22.0,2.0
5,1995,2989,8.0,0.0
6,1990,3215,15.0,0.0
7,1997,3651,17.0,0.0
8,1998,4131,16.0,2.0
9,1999,5385,6.0,0.0


**Question 3.5** (10 points) Show the name of the specie with most incidents and the related number of incidents. The result cannot be an unknow bird.

In [52]:
# question 3.5
#df = pd.read_csv('faa-wildlife-strikes.csv')
df3 = df[~(df['SPECIES'].str.contains('Unknown',na=False))]
df3 = df3['SPECIES'].groupby(df3['SPECIES']).count().sort_values(ascending = False).head(1)
df3

SPECIES
Mourning dove    7653
Name: SPECIES, dtype: int64

**Question 3.6** (10 points) Show the top 5 species present on records with engine damage (from engine 1 to 4). For this exercise you can consider unkown species.

In [58]:
# question 3.6
df['DAM_ENG'] = df[['DAM_ENG1','DAM_ENG2','DAM_ENG3','DAM_ENG4']].sum(axis=1)

df4 = df[(df.DAM_ENG > 0)]
df4 = df[['SPECIES','DAM_ENG']].groupby(['SPECIES']).count().sort_values(by = ['DAM_ENG'],ascending=False)        
df4[0:5]

,DAM_ENG
SPECIES,
Unknown bird - medium,38531
Unknown bird - small,32981
Mourning dove,7653
Gulls,6580
Unknown bird,6400
